# Library

In [1]:
import numpy as np
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import torch
import argparse
import time
import pickle

#from src.self_awareness.networks import utils
#from src.self_awareness.learning.tf_cnn_auxiliary_gp import Model
from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

import sys
sys.path.append('..')

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


# Check GPU

In [2]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


# Set Arguments

In [3]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=800, help='size of mini batch')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/dual_resnet_torch', help='model directory')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
parser.add_argument('--enable_ros', type=bool, default=False, help='put data into ros')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')

sys.argv = ['']
args = parser.parse_args()

if args.enable_ros:
    rospy.init_node('global_localization_tf_broadcaster_cnn')

# Load Dataset

In [4]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet, cnn_auxiliary
from torchlib.cnn_auxiliary import normalize, denormalize, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.test_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate', sampling_rate=1)
if len(args.train_dataset)>7:
    [args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
    torch.save([args.norm_mean, args.norm_std], *args.norm_tensor)
    print('Save norm and std:',*args.norm_tensor)
else:
    [args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
    print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

100%|██████████| 25367/25367 [00:59<00:00, 423.62it/s]


Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [5]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, BaseModule
        
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        
    def forward(self, *args):
        outputs = []
        for input_data in args:
            dense_feat = self.backbone(input_data)
            output, feature_t, feature_r = self.nn(dense_feat)
            outputs += [output]
        if len(args)>1:
            return outputs
        else:
            return output, feature_t, feature_r
        
class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args, is_training=True, mixed_precision=True):
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        self.mixed_precision = mixed_precision
        if self.mixed_precision:
            self.scaler = GradScaler()
        if is_training:
            self.optimizer = optim.Adam(self.model.parameters(), 
                                        lr=args.learning_rate, 
                                        weight_decay=args.weight_decay)
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
        else:
            self.disable_requires_grad(self.model)
    
    def train(self,x0, x1, y0, y1):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x0,x1,y0,y1 = x0.cuda(),x1.cuda(),y0.cuda(),y1.cuda()
        if args.is_normalization:
            y0, y1 = [normalize(y,self.norm_mean, self.norm_std) for y in [y0,y1]]
            
        # Step 2: training
        assert trainer.model.training == True
        
        if self.mixed_precision:
            with autocast():
                global_loss, consistent_loss = self._loss(x0, x1, y0, y1)
                single_loss = global_loss + consistent_loss
                
            self.scaler.scale(single_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            global_loss, consistent_loss = self._loss(x0, x1, y0, y1)
            single_loss = global_loss + consistent_loss
            
            single_loss.backward()
            self.optimizer.step()
        batch_time = time.time() - start
        
        return float(single_loss), float(global_loss), float(consistent_loss), batch_time
    
    def _loss(self,x0, x1, y0, y1):
        # target relative
        relative_target_normed = get_relative_pose(y0, y1)
        # forward output
        global_output0,global_output1 = self.model(x0, x1)
        # output relative
        relative_consistence = get_relative_pose(global_output0,global_output1)
        
        # target loss
        global_loss = translational_rotational_loss(pred=global_output1, gt=y1, \
                                                    lamda=args.lamda_weights)
        # relative loss
        geometry_consistent_loss = translational_rotational_loss(pred=relative_consistence, \
                                                                 gt=relative_target_normed, \
                                                                 lamda=args.lamda_weights)        
        
        return global_loss, geometry_consistent_loss
    
    def eval_forward(self,x,y,output_denormalize = True):
        # Step 1: get data
        x,y = x.cuda(),y.cuda()
        
        # Step 2: forward
        assert trainer.model.training == False
        
        if self.mixed_precision:
            with autocast():
                output,_,_ = self.model(x)
            self.scaler.scale(output)
        else:
            output,_,_ = self.model(x)

        if args.is_normalization and output_denormalize:
            output = denormalize(output, self.norm_mean, self.norm_std)
            
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        trans_prediction, rot_prediction = torch.split(output, [3, 4], dim=1)
        return trans_prediction, rot_prediction, trans_target, rot_target

In [6]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,mixed_precision=True,is_training=False)
trainer.load_model('pretrained.pth')

Successfully loaded model to TITAN Xp.


# Initialize

In [7]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

# Evaluate

In [8]:
trainer.model.eval()

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    trans_pred, rot_pred, trans_gt, rot_gt = trainer.eval_forward(x,y)
    
    # transform data
    trans_pred = trans_pred.cpu().numpy()
    rot_pred = rot_pred.cpu().numpy()
    trans_gt = trans_gt.cpu().numpy()
    rot_gt = rot_gt.cpu().numpy()
    
    end = time.time()
    
    if args.enable_ros:
        for i in range(y.shape[0]):
            br.sendTransform(trans_pred[i],rot_pred[i], rospy.Time.now(),"estimation", "world")
            br.sendTransform(trans_gt[i],rot_gt[i],rospy.Time.now(), "gt", "world")

            timestamp = rospy.Time.now()

            gt_msg = Odometry()
            gt_msg.header.frame_id = 'world'
            gt_msg.header.stamp = timestamp
            gt_msg.child_frame_id = 'base_link'
            gt_msg.pose.pose.position.x = trans_gt[i][0]
            gt_msg.pose.pose.position.y = trans_gt[i][1]
            gt_msg.pose.pose.position.z = trans_gt[i][2]
            gt_msg.pose.pose.orientation.x = rot_gt[i][0]
            gt_msg.pose.pose.orientation.y = rot_gt[i][1]
            gt_msg.pose.pose.orientation.z = rot_gt[i][2]
            gt_msg.pose.pose.orientation.w = rot_gt[i][3]

            bridge = CvBridge()

            bird_view_img_msg = bridge.cv2_to_imgmsg(x[i][0], encoding="passthrough") # 1x300x300 -> 300x300
            stamp_now = rospy.Time.now()
            bird_view_img_msg.header.stamp = stamp_now

            bird_view_pub.publish(bird_view_img_msg)

            rospy.sleep(.0)

            count += 1
    else:
        count += y.shape[0]
    
    trans_preds += [x for x in trans_pred]
    rot_preds += [x for x in rot_pred]
    trans_gts += [x for x in trans_gt]
    rot_gts += [x for x in rot_gt]

    trans_error = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
    rot_error_1 = np.arccos(np.sum(np.multiply(rot_pred,rot_gt),axis=1))/math.pi*180
    rot_error_2 = np.arccos(np.sum(np.multiply(rot_pred,-rot_gt),axis=1))/math.pi*180
    rot_error = np.minimum(rot_error_1,rot_error_2)

    trans_errors += [x for x in trans_error]
    rot_errors += [x for x in rot_error]

    total_trans_error += np.sum(trans_error)
    total_rot_error += np.sum(rot_error)
    
    display = 1

    if b % display == 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             (b+1)*args.batch_size,
            len(dataloader)*args.batch_size,
            total_trans_error / count,
            total_rot_error / count,
            end - start))
    

800/184000, translation error = 7.099, rotation error = 4.380, time/batch = 7.737
1600/184000, translation error = 6.167, rotation error = 3.596, time/batch = 2.352
2400/184000, translation error = 8.481, rotation error = 3.763, time/batch = 2.309
3200/184000, translation error = 7.412, rotation error = 3.649, time/batch = 2.169
4000/184000, translation error = 6.660, rotation error = 4.271, time/batch = 2.173
4800/184000, translation error = 6.153, rotation error = 4.276, time/batch = 2.176
5600/184000, translation error = 5.753, rotation error = 4.414, time/batch = 2.176
6400/184000, translation error = 5.454, rotation error = 4.407, time/batch = 2.199
7200/184000, translation error = 5.359, rotation error = 4.393, time/batch = 2.192
8000/184000, translation error = 5.139, rotation error = 4.274, time/batch = 2.173
8800/184000, translation error = 4.862, rotation error = 4.265, time/batch = 2.191
9600/184000, translation error = 4.621, rotation error = 4.297, time/batch = 2.193
10400

79200/184000, translation error = 10.250, rotation error = 6.313, time/batch = 2.240
80000/184000, translation error = 10.193, rotation error = 6.289, time/batch = 2.232
80800/184000, translation error = 10.120, rotation error = 6.259, time/batch = 2.236
81600/184000, translation error = 10.061, rotation error = 6.253, time/batch = 2.249
82400/184000, translation error = 10.020, rotation error = 6.247, time/batch = 2.237
83200/184000, translation error = 9.960, rotation error = 6.223, time/batch = 2.228
84000/184000, translation error = 9.908, rotation error = 6.212, time/batch = 2.203
84800/184000, translation error = 9.920, rotation error = 6.203, time/batch = 2.231
85600/184000, translation error = 9.868, rotation error = 6.178, time/batch = 2.232
86400/184000, translation error = 9.800, rotation error = 6.157, time/batch = 2.235
87200/184000, translation error = 9.759, rotation error = 6.130, time/batch = 2.210
88000/184000, translation error = 9.747, rotation error = 6.136, time/b

156800/184000, translation error = 11.341, rotation error = 6.973, time/batch = 2.256
157600/184000, translation error = 11.369, rotation error = 6.991, time/batch = 2.240
158400/184000, translation error = 11.490, rotation error = 6.992, time/batch = 2.248
159200/184000, translation error = 11.853, rotation error = 7.024, time/batch = 2.224
160000/184000, translation error = 11.837, rotation error = 7.078, time/batch = 2.228
160800/184000, translation error = 11.813, rotation error = 7.091, time/batch = 2.236
161600/184000, translation error = 11.857, rotation error = 7.134, time/batch = 2.236
162400/184000, translation error = 11.858, rotation error = 7.196, time/batch = 2.234
163200/184000, translation error = 11.845, rotation error = 7.331, time/batch = 2.226
164000/184000, translation error = 11.835, rotation error = 7.321, time/batch = 2.235
164800/184000, translation error = 11.830, rotation error = 7.307, time/batch = 2.232
165600/184000, translation error = 11.812, rotation er

In [9]:
import scipy.io as sio

sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

median translation error = 3.699
median rotation error = 3.952
mean translation error = 11.882
mean rotation error = 7.628


In [10]:
def evaluate(trans_errors,rot_errors):
    t = [14301,7008,12852,9567,13580,14835,7114,12683]
    for i in range(len(t)):
        if i >0:
            t[i] += t[i-1]
    trans_errors_month = list()
    trans_errors_month.append(trans_errors[:t[0]])
    trans_errors_month.append(trans_errors[t[0]:t[1]])
    trans_errors_month.append(trans_errors[t[1]:t[2]])
    trans_errors_month.append(trans_errors[t[2]:t[3]])
    trans_errors_month.append(trans_errors[t[3]:t[4]])
    trans_errors_month.append(trans_errors[t[4]:t[5]])
    trans_errors_month.append(trans_errors[t[5]:t[6]])
    trans_errors_month.append(trans_errors[t[6]:])

    rot_errors_month = list()
    rot_errors_month.append(rot_errors[:t[0]])
    rot_errors_month.append(rot_errors[t[0]:t[1]])
    rot_errors_month.append(rot_errors[t[1]:t[2]])
    rot_errors_month.append(rot_errors[t[2]:t[3]])
    rot_errors_month.append(rot_errors[t[3]:t[4]])
    rot_errors_month.append(rot_errors[t[4]:t[5]])
    rot_errors_month.append(rot_errors[t[5]:t[6]])
    rot_errors_month.append(rot_errors[t[6]:])
    
    print('================== median translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("median translation error = {:.3f}".format(np.median(trans_errors_i)))
        
    print('================== median rotation error ==================')
    for rot_errors_i in rot_errors_month:
        print("median rotation error = {:.3f}".format(np.median(rot_errors_i)))
    
    print('================== mean translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("mean translation error = {:.3f}".format(np.mean(trans_errors_i)))
        
    print('================== mean rotation error ==================')  
    for rot_errors_i in rot_errors_month:
        print("mean rotation error = {:.3f}".format(np.mean(rot_errors_i)))
        
evaluate(trans_errors,rot_errors)

================== median translation error ==================
median translation error = 3.038
median translation error = 3.589
median translation error = 3.266
median translation error = 2.814
median translation error = 3.277
median translation error = 4.014
median translation error = 3.012
median translation error = 4.092
================== median rotation error ==================
median rotation error = 3.204
median rotation error = 4.638
median rotation error = 3.376
median rotation error = 3.608
median rotation error = 3.335
median rotation error = 3.751
median rotation error = 4.103
median rotation error = 4.296
================== mean translation error ==================
mean translation error = 6.290
mean translation error = 5.723
mean translation error = 6.602
mean translation error = 11.186
mean translation error = 16.765
mean translation error = 15.877
mean translation error = 3.778
mean translation error = 13.122
================== mean rotation error ==================
me